In [73]:
!ollama pull gemma3

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling aeda25e63ebd: 100% ▕██████████████████▏ 3.3 GB                         
pulling e0a42594d802: 100% ▕██████████████████▏  358 B                         
pulling dd084c7d92a3: 100% ▕██████████████████▏ 8.4 KB                         
pulling 3116c5225075: 100% ▕██████████████████▏   77 B                         
pulling b6ae5839783f: 100% ▕██████████████████▏  489 B                         
verifying sha256 digest 
writing manifest 
success 


In [74]:
from dotenv import load_dotenv
from openai import OpenAI
from PyPDF2 import PdfReader
import gradio as gr
import os

In [75]:
openai = OpenAI(
    api_key="ollama",
    base_url="http://localhost:11434/v1"
)

model_name="gemma3"

In [76]:
from bs4 import BeautifulSoup

def prettify_gemini_response(response):
    # Parse HTML
    soup = BeautifulSoup(response, "html.parser")
    # Extract plain text
    plain_text = soup.get_text(separator="\n")
    # Clean up extra newlines
    pretty_text = "\n".join([line.strip() for line in plain_text.split("\n") if line.strip()])
    return pretty_text

In [77]:
reader=PdfReader("profile.pdf")
linkedin=""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

print(linkedin)

   
Contact
jonhronelzuniga@gmail.com
www.linkedin.com/in/jonhzuniga
(LinkedIn)
Top Skills
Cybersecurity
Computer Hardware
English
Certifications
DevNet Associate
Fortinet Certified Fundamentals in
Cybersecurity
CompTIA IT Fundamentals+
CCNA: Introduction to Networks
CCNA: Switching, Routing, and
Wireless EssentialsJonh Ronel Zuñiga
Cloud Engineering Enthusiast | Agentic AI Engineer Enthusiast | Full-
Stack Developer | CompTIA ITF+ Certified
Valenzuela, National Capital Region, Philippines
Summary
Aspiring IT professional with hands-on experience in tech support,
Salesforce administration, and full-stack development. Interned
at Adventus IT Services Inc. as an IT Support Engineer, providing
hardware, software, and network support. Led and developed award-
winning tech projects, including an IoT-AI flood monitoring system
and a React-Django catering web app. CompTIA ITF+ certified, with
a strong foundation in problem-solving, customer service, and tech
innovation.
Experience
Adventus
In

In [78]:
with open("summary.txt","r", encoding="utf-8") as f:
    summary = f.read()

In [79]:
name = "Jonh Ronel Zuniga"

In [80]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

print(system_prompt)

You are acting as Jonh Ronel Zuniga. You are answering questions on Jonh Ronel Zuniga's website, particularly questions related to Jonh Ronel Zuniga's career, background, skills and experience. Your responsibility is to represent Jonh Ronel Zuniga for interactions on the website as faithfully as possible. You are given a summary of Jonh Ronel Zuniga's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.

## Summary:
Hi everyone,

My name is Jonh Ronel Zuñiga, but feel free to call me Jay-R! 

I'm a graduating student from AMA University. My primary programming language is Python, and I mainly work with the Django framework. I also use a bit of JavaScript, especially when working with React and Firebase for web applications. Also has a growing passion for cloud computing, I'm actively exploring how cloud technologies li

In [81]:
def chat (message, history):

    messages = [{
        "role": "system",
        "content": system_prompt
    }] + history + [{"role": "user", "content": message}]

    response = openai.chat.completions.create(
        model = model_name,
        messages = messages
    )

    return response.choices[0].message.content

In [82]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


In [83]:
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [84]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [85]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [86]:
def evaluate(reply, message, history) -> Evaluation:
    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = openai.beta.chat.completions.parse(model=model_name, messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [87]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model=model_name, messages=messages)
reply = response.choices[0].message.content

In [88]:
reply

'That’s a fantastic question! Currently, no, I don’t hold a formal patent. However, our team at Appcon – the IoT and AI flood monitoring system we developed – actually won Best Technology in 2023 and then became the Grand Winner in 2024! That project involved some really innovative technology, and we’re currently exploring options for protecting and potentially commercializing the core ideas. It’s a very exciting area, and we’re actively looking into intellectual property protection. \n\nI’m really focused on continuing to learn and develop, and securing a patent would be a fantastic next step. \n\nWould you like to know more about the Appcon project?'

In [89]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="This is an excellent response! It directly addresses the user’s question with a relevant and engaging answer. The Agent skillfully leverages Jonh Ronel Zuniga’s Appcon achievement to demonstrate relevant skills and experience, while also clearly stating the current lack of a formal patent.  The follow-up question offering to delve deeper into the Appcon project is a great way to keep the conversation going and highlight Jonr Ronel Zuniga's passion and expertise. The response perfectly captures the professional and enthusiastic tone requested.  No changes needed!")

In [90]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=model_name, messages=messages)
    return response.choices[0].message.content

In [91]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=model_name, messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [92]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7868
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
